In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import sys
import time
import pickle
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report 
from sklearn.metrics import precision_score , recall_score
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt 
import tensorflow as tf


def preprocess(gtzan_spectrograms, gtzan_labels):
    #trainingData = np.concatenate((benchmark_spectrograms, gtzan_spectrograms), axis=0)
    #label = np.concatenate((benchmark_labels, gtzan_labels), axis=0)
    trainingData = gtzan_spectrograms
    label = gtzan_labels
    print(len(trainingData))
    print(len(label))
    one_hot_label = []
    for k in label:
        if k == 0:
            one_hot_label.append([1, 0, 0, 0, 0])
        if k == 1:
            one_hot_label.append([0, 1, 0, 0, 0])
        if k == 2:
            one_hot_label.append([0, 0, 1, 0, 0])
        if k == 3:
            one_hot_label.append([0, 0, 0, 1, 0])
        if k == 4:
            one_hot_label.append([0, 0, 0, 0, 1])

    x_train, x_validation, y_train, y_validation = train_test_split(trainingData, one_hot_label,train_size=4070, test_size=453,shuffle=True)
    x_test, x_validation, y_test, y_validation = train_test_split(x_validation,y_validation, train_size=226, test_size=227, shuffle=True)

    test_label = []
    for i in list(y_test):
        if i == [1, 0, 0, 0, 0]:
            test_label.append(0)
        if i == [0, 1, 0, 0, 0]:
            test_label.append(1)
        if i == [0, 0, 1, 0, 0]:
            test_label.append(2)
        if i == [0, 0, 0, 1, 0]:
            test_label.append(3)
        if i == [0, 0, 0, 0, 1]:
            test_label.append(4)

    x_train = np.array(x_train)
    x_validation = np.array(x_validation)
    x_test = np.array(x_test)
    y_train = np.array(y_train)
    y_validation = np.array(y_validation)
    y_test = np.array(y_test)

    return x_train, y_train, x_validation, y_validation, x_test, y_test,test_label

# Use the following to load spectrograms:
# modify name_suffix to choose dataset
name_suffix = '_3_1024'
#benchmark_spectrograms = pickle.load(open('drive/My Drive/mgc_dataset/Copy of benchmark_spectrograms' + name_suffix + '.p', 'rb'))
gtzan_spectrograms = pickle.load(open('drive/My Drive/mgc_dataset/gtzan_spectrograms' + name_suffix + '.p', 'rb'))
#benchmark_labels = pickle.load(open('drive/My Drive/mgc_dataset/benchmark_labels_3.p', 'rb'))
gtzan_labels = pickle.load(open('drive/My Drive/mgc_dataset/gtzan_labels_3_1024.p', 'rb'))

x_train, y_train, x_validation, y_validation, x_test, y_test,test_label = preprocess(gtzan_spectrograms, gtzan_labels)
x_train = np.expand_dims(x_train, -1)
x_validation = np.expand_dims(x_validation, -1)
x_test = np.expand_dims(x_test, -1)

input_shape = (513, 128, 1)
#input_shape = (257, 858, 1)
num_category = 5
model = Sequential()
model.add(layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Flatten())
model.add(layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(num_category, activation='softmax'))
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history =model.fit(x_train, y_train, batch_size=16, epochs=20, validation_data=(x_validation, y_validation), callbacks=[callback])

score, acc = model.evaluate(x_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

plt.plot(history.history['acc'], label='accuracy')
plt.plot(history.history['val_acc'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

#evaluate the data set.
predictTesting = model.predict_classes(np.asarray(x_test))
print("Testing dataset confusion matrix: \n", confusion_matrix(test_label, predictTesting))
print('Testing dataset Accuracy Score: \n', accuracy_score(test_label, predictTesting))
print('Testing dataset Report : \n',classification_report(test_label, predictTesting))




4523
4523
Train on 4070 samples, validate on 227 samples
Epoch 1/20
4070/4070 [==============================] - 268s 66ms/sample - loss: 1.3877 - acc: 0.4774 - val_loss: 1.1684 - val_acc: 0.6211
Epoch 2/20
4070/4070 [==============================] - 266s 65ms/sample - loss: 1.1690 - acc: 0.6477 - val_loss: 1.0356 - val_acc: 0.7225
Epoch 3/20
4070/4070 [==============================] - 269s 66ms/sample - loss: 1.1216 - acc: 0.7039 - val_loss: 1.0913 - val_acc: 0.7621
Epoch 4/20
4070/4070 [==============================] - 268s 66ms/sample - loss: 1.0935 - acc: 0.7420 - val_loss: 1.0803 - val_acc: 0.7577
Epoch 5/20
4070/4070 [==============================] - 267s 66ms/sample - loss: 1.0162 - acc: 0.7776 - val_loss: 1.0816 - val_acc: 0.7841
Epoch 6/20
4070/4070 [==============================] - 265s 65ms/sample - loss: 0.9683 - acc: 0.8170 - val_loss: 1.0678 - val_acc: 0.8062
Epoch 7/20
1792/4070 [============>.................] - ETA: 2:26 - loss: 0.9172 - acc: 0.8304